# Programación declarativa @ URJC
# Programación funcional
## Curso 21-22, convocatoria ordinaria (27 de octubre de 2021)
## Campus de Vicálvaro


# Definiciones auxiliares

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

### Algunas definiciones de tipos y funciones auxiliares

In [ ]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

object Tree{
    def void[A]: Tree[A] = Empty()
    def leaf[A](a: A): Node[A] = Node(Empty(), a, Empty())
    def right[A](a: A, tree: Tree[A]): Node[A] = Node(Empty(), a, tree)
    def left[A](tree: Tree[A], a: A): Node[A] = Node(tree, a, Empty())
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = Node(left, a, right)
}

import Tree._

def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    tree match {
        case Empty() => empty
        case Node(left, root, right) => node(foldTree(left)(empty)(node), root, foldTree(right)(empty)(node))
    }

In [ ]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
 
        // Returns the number of elements of this list
        def length: Int
        def reverse: List[A]
    }
}

### Definiciones auxiliares sobre la correspondencia Curry-Howard

In [ ]:
type Not[P] = P => Nothing

# Ejercicio 1
__(1,5 puntos)__

__a) (0,5 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar la siguiente tautología de la lógica proposicional intuicionista: 

$\neg p \rightarrow \neg\neg\neg p$

In [ ]:
def proof[P]: Not[P] => Not[Not[Not[P]]] = 
    (notP: P => Nothing) => 
        (f: (P => Nothing) => Nothing) => 
            f(notP) : Nothing

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$(\neg q \rightarrow \neg p) \rightarrow (p \rightarrow q)$

Supóngase para ello que la ley del tercio excluso se cumple para la variable proposicional $q$, es decir, que la fórmula $q \vee \neg q$  puede utilizarse como premisa.


In [ ]:
def proof[P, Q](middle: Either[Q, Not[Q]]): (Not[Q] => Not[P]) => (P => Q) = 
    (f: Not[Q] => Not[P]) => (p: P) => 
        middle match {
            case Left(q: Q) => 
                q : Q
            case Right(notQ: Not[Q]) => 
                f(notQ)(p) : Q
        }

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$(\neg q \rightarrow \neg p) \rightarrow (p \rightarrow q)$

Supóngase para ello que la ley de la doble negación se cumple para la variable proposicional $q$, es decir, que la fórmula $\neg \neg q \rightarrow q$  puede utilizarse como premisa.


In [ ]:
def proof[P, Q](dn: Not[Not[Q]] => Q): (Not[Q] => Not[P]) => (P => Q) = 
    (f: Not[Q] => Not[P]) => (p: P) => 
        dn((notQ: Not[Q]) => f(notQ)(p) : Nothing) : Q

# Ejercicio 2
__(1 punto)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$(1+1)^X \cong Boolean^X$

A continuación se muestran unos casos de prueba de este isomorfismo para $X=Int$:

In [ ]:
class IsoTest(
    from: (Int => Either[Unit, Unit]) => Int => Boolean, 
    to: (Int => Boolean) => Int => Either[Unit, Unit]
) extends FlatSpec with Matchers{
    
    val f: Int => Either[Unit, Unit] = 
        i => if (i % 2 == 0) Left(()) else Right(())
    
    val g: Int => Boolean = 
        _ % 2 == 0
    
    "from-to" should "work" in {
        from(to(g))(0) shouldBe g(0)
        from(to(g))(1) shouldBe g(1)
        from(to(g))(2) shouldBe g(2)
        from(to(g))(3) shouldBe g(3)
    }
    
    "to-from" should "work" in {
        to(from(f))(0) shouldBe f(0)
        to(from(f))(1) shouldBe f(1)
        to(from(f))(2) shouldBe f(2)
        to(from(f))(3) shouldBe f(3)
    }
}

In [ ]:
def from[X](l: X => Either[Unit, Unit]): X => Boolean = 
    x => l(x) match {
        case Left(()) => true
        case Right(()) => false
    }

In [ ]:
def to[X](l: X => Boolean): X => Either[Unit, Unit] = 
    x => if (l(x)) Left(()) else Right(())

In [ ]:
run(new IsoTest(from[Int], to[Int]))

# Ejercicio 3
__(3 puntos)__

La función `slice` recibe una lista de valores de tipo `X` y un rango de posiciones, y devuelve una lista con los elementos comprendidos dentro de ese rango. El comportamiento de la función se ilustra en el siguiente test unitario, donde la función `slice` se encuentra particularizada para el tipo `X=Int`:


In [ ]:
class TestSlice(
    slice: List[Int] => (Int, Int) => List[Int]
) extends FlatSpec with Matchers{
    "slice" should "work" in {
        slice(List())(0,3) shouldBe List()
        slice(List(1,2,3,4))(5,6) shouldBe List()
        slice(List(1,2,3,4))(0,2) shouldBe List(1,2,3)
        slice(List(1,2,3,4))(0,6) shouldBe List(1,2,3,4)
        slice(List(1,2,3,4))(1,3) shouldBe List(2,3,4)
        slice(List(1,2,3,4))(1,2) shouldBe List(2,3)
    }
}

__a) (1,5 puntos)__ Implementa la función `slice` mediante recursión final (o de cola).

In [ ]:
def slice[A](list: List[A])(from: Int, to: Int): List[A] = {
    def auxSlice(aux: List[A])(out: List[A], idx: Int): List[A] = 
        aux match {
            case e :: tail if from <= idx && idx <= to => 
                auxSlice(tail)(e :: out, idx + 1)
            case _ :: tail if idx < from => 
                auxSlice(tail)(out, idx + 1)
            case _ => 
                out
        }
    
    auxSlice(list)(Nil, 0).reverse
}

In [ ]:
run(new TestSlice(slice))

__b) (1,5 puntos)__ Implementa la función `slice` con `foldLeft`.

In [ ]:
def slice[A](list: List[A])(from: Int, to: Int): List[A] = 
    list.foldLeft((Nil: List[A], 0)){
        case ((out, idx), e) if from <= idx && idx <= to => 
            (e :: out, idx + 1)
        case ((out, idx), _) => 
            (out, idx + 1)
    }._1.reverse

In [ ]:
run(new TestSlice(slice))

# Ejercicio 4
__(3 puntos)__

Considérese una función que dado un árbol binario devuelve el camino más largo desde la raíz a sus hojas. Si existen varios caminos con la misma longitud máxima, la función devuelve uno cualquiera de ellos. Por ejemplo:

In [ ]:
class TestLongestPath(longest: Tree[Int] => List[Int]) extends FlatSpec with Matchers{
    "longest path" should "work" in {
        longest(void) shouldBe 
            List()
        
        longest(left(left(right(3,right(2,leaf(1))), 4), 5)) shouldBe 
            List(5,4,3,2,1)
        
        longest(node(left(leaf(4), 1), 0, 
                     node(leaf(3), 2, right(2, right(4, leaf(5)))))) shouldBe 
            List(0, 2, 2, 4, 5)
        
        longest(node(left(right(0, leaf(1)), 2), 3, node(left(leaf(5), 4), 9, leaf(7)))) should 
            (equal(List(3, 2, 0, 1)) or equal(List(3, 9, 4, 5)))
    }
}

__a) (1,5 puntos)__ Implementa la función `longestPath` recursivamente. La implementación podrá hacer uso del método `length` de la clase `List[A]`.

In [ ]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    tree match {
        case Empty() => List()
        case Node(left, root, right) => 
            val longestLeft: List[A] = longestPath(left)
            val longestRight: List[A] = longestPath(right)
            root :: (if (longestRight.length > longestLeft.length) longestRight
                     else longestLeft)
    }

In [ ]:
run(new TestLongestPath(longestPath))

__b) (1,5 puntos)__ Implementa la función `longestPath` mediante la función de orden superior `foldTree`, __sin__ hacer uso de la función `length`.

In [ ]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    foldTree(tree)((0, List[A]())){
        case ((l, longestLeft), root, (r, longestRight)) => 
            if (l > r) (l+1, root :: longestLeft) 
            else (r+1, root :: longestRight)
    }._2

In [ ]:
run(new TestLongestPath(longestPath))

# Ejercicio 5
__(1,5 puntos)__

El patrón de diseño de divide y vencerás puede describirse de manera simplificada en los siguientes términos:
* El patrón se aplica a problemas de tipo `P` que devuelven soluciones de tipo `S`
* Un problema de tipo `P` puede ser atómico, es decir, indivisible, o descomponible en dos subproblemas del mismo tipo `P` 
* Un problema atómico se puede resolver directamente
* Un problema descomponible se puede resolver mediante la composición de las soluciones de sus subproblemas


__a) (1 punto)__ Implementa esta versión simplificada del patrón de divide y vencerás mediante la siguiente función de orden superior `dyv`, donde: 
* Los parámetros `P` y `S` representan el tipo del problema y de la solución, respectivamente
* El parámetro `problem` representa el problema a resolver
* La función `decompose` devuelve un valor de tipo `Left` en caso de que el problema sea atómico, o bien un valor de tipo `Right` en caso de que el problema sea descomponible
* La función `atomic` resuelve directamente un problema atómico de tipo `P`
* La función `compose` combina dos soluciones para obtener una solución global

In [ ]:
def dyv[P, S](problem: P)(
              decompose: P => Either[P, (P, P)],
              atomic: P => S,
              compose: (S, S) => S): S = 
    decompose(problem) match {
        case Left(base) => atomic(base)
        case Right((problem1, problem2)) => 
            compose(dyv(problem1)(decompose, atomic, compose),
                    dyv(problem2)(decompose, atomic, compose))
    }

__b) (0,5 puntos)__ A continuación se muestra una implementación ad-hoc del algoritmo de ordenación por mezcla: 

In [ ]:
def merge(array1: Array[Int], array2: Array[Int]): Array[Int] = 
    (array1, array2) match {
        case (Array(), Array()) => Array.empty
        case (Array(), ys2)     => ys2
        case (xs2, Array())     => xs2
        case (xs1@Array(x, tail1@_*), ys1@Array(y, tail2@ _*)) =>
            if (x < y) x +: merge(tail1.toArray, ys1)
            else y +: merge(xs1, tail2.toArray)
    }

In [ ]:
def mergeSort(numbers: Array[Int]): Array[Int] = 
    if (numbers.length <= 1) numbers
    else merge(mergeSort(numbers.slice(0, numbers.length/2)), 
               mergeSort(numbers.slice(numbers.length/2, numbers.length)))

Este algoritmo puede considerarse una instancia del esquema de divide y vencerás, siendo el tipo del problema `Array[Int]` y el tipo de la solución igualmente `Array[Int]`. Obsérvese que: 
* Se puede distinguir entre problemas atómicos (arrays con una longitud menor o igual a uno) y problemas descomponibles (con una longitud mayor que uno)
* Un problema atómico se resuelve directamente devolviendo el mismo array de entrada
* Un problema descomponible se resuelve mezclando los dos arrays ordenados que se obtienen tras descomponer el array de entrada en dos partes y ordenarlos de manera independiente. 

__Se pide__ reimplementar el algoritmo de ordenación por mezcla utilizando la función `dyv` del apartado anterior. La implementación podrá hacer uso de las funciones auxiliares utilizadas en la implementación ad-hoc (en particular, `merge` y `_.slice`). 

In [ ]:
def mergeSort(numbers: Array[Int]): Array[Int] = {

    def split(numbers: Array[Int]): Either[Array[Int], (Array[Int], Array[Int])] = 
        if (numbers.length <= 1) Left(numbers)
        else Right((numbers.slice(0, numbers.length/2), 
                    numbers.slice(numbers.length/2, numbers.length)))

    dyv(numbers)(split, numbers => numbers, merge)
}

In [ ]:
class TestMergeSort(sort: Array[Int] => Array[Int]) extends FlatSpec with Matchers{
    "merge sort" should "work" in {
        sort(Array(8,7,6,5,4,3,2,1)) shouldBe Array(1,2,3,4,5,6,7,8)
        sort(Array()) shouldBe Array()
        sort(Array(1)) shouldBe Array(1)
        sort(Array(5,3,4,7,1,2,8,6)) shouldBe Array(1,2,3,4,5,6,7,8)
    }
}

In [ ]:
run(new TestMergeSort(mergeSort))